In [1]:
# import the necessary packages
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from loader_util.preprocessing import ImageToArrayPreprocessor, \
    AspectAwarePreprocessor, \
    MeanSubtractionPreProcessor
from loader_util.datasets import SimpleDatasetLoader
from loader_util.nn.conv import FCHeadNet
##
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop, SGD
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from imutils import paths
import numpy as np
import os

In [4]:
args = {
    'dataset':r"C:\Users\mhasa\Google Drive\Tutorial "
              r"Corner\PYTH\DeepLearning\DeepLearning-DL4CV\ImageDatasets"
              r"\cellImages3"}

In [5]:
# construct the image generator
aug = ImageDataGenerator(rotation_range=30,
                         width_shift_range=0.1,
                         height_shift_range=0.1,
                         shear_range=0.2,
                         zoom_range=0.2, 
                         horizontal_flip=True,
                         vertical_flip=True,
                         fill_mode='nearest')

In [6]:
# grab the list of images that we'll be describing then extract the class 
# label names from the imagePaths
imagePaths = list(paths.list_images(args['dataset']))
classNames = [pt.split(os.path.sep)[-2] for pt in imagePaths]
classNames = [str(x) for x in np.unique(classNames)]

In [7]:
# initialise the image preprocessor
msp = MeanSubtractionPreProcessor()
aap = AspectAwarePreprocessor(224,224)
iap = ImageToArrayPreprocessor()

# load the dataset from disk then scale the raw pixels
sdl = SimpleDatasetLoader(preprocessors=[msp, aap, iap])
data, labels = sdl.load(imagePaths, verbose=500)
#data = data.astype('float') / 255.0

[INFO] processed 500/1728
[INFO] processed 1000/1728
[INFO] processed 1500/1728


In [8]:
# partition the data into training and test splits
trainx, testx, trainy, testy = train_test_split(data,
                                                labels,
                                                test_size=0.25,
                                                random_state=42)

# convert labels from integers into vectors
le = LabelBinarizer()
trainy = le.fit_transform(trainy)
testy = le.transform(testy)

In [9]:
# load the VGG16 network ensuring the head FC layer sets are left off
baseModel = VGG16(weights='imagenet', include_top=False, 
                  input_tensor=Input(shape=(224, 224, 3))) # type: Model

# initialise the new head of the network, a set of FC layers followed by 
# softmax
headModel = FCHeadNet.builld(baseModel, len(classNames), D=256) # type: Model

# place the head FC model on top of the base model - this will be the actual
# model to train
model = Model(inputs=baseModel.input, outputs=headModel)

(None, 7, 7, 512)


In [10]:
# loop over all the layers in the base model and freeze them
for layer in baseModel.layers:
    layer.trainable = False

In [11]:
# compile and optimise model
opt = RMSprop(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# train the head for few epochs
model.fit_generator(aug.flow(trainx, trainy, batch_size=32),
                    validation_data=(testx, testy),
                    epochs=25,
                    steps_per_epoch=len(trainx) // 32,
                    verbose=1)

Epoch 1/25
40/40 [==============================] - 134s 3s/step - loss: 7.4086 - accuracy: 0.4913 - val_loss: 5.7224 - val_accuracy: 0.6366

In [12]:
# evaluate the network
preds = model.predict(testx, batch_size=32)
print(classification_report(testy.argmax(axis=1),
                            preds.argmax(axis=1),
                            target_names=classNames))

              precision    recall  f1-score   support

         3T3       0.95      1.00      0.97       156
       MG 63       1.00      0.95      0.97       129
        hASC       1.00      0.99      1.00       147

    accuracy                           0.98       432
   macro avg       0.98      0.98      0.98       432
weighted avg       0.98      0.98      0.98       432



In [13]:
# now that FC layers have been trained, lets unfreeze the final set of CONV 
# layers and make them trainable
for layer in baseModel.layers[15:]:
    layer.trainable = True

In [14]:
# for changes to take effect we need to recompile model
opt = SGD(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.fit_generator(aug.flow(trainx, trainy, batch_size=32),
                    validation_data=(testx, testy),
                    epochs=100,
                    steps_per_epoch=len(trainx) // 32,
                    verbose=1)

Epoch 1/100
40/40 [==============================] - 95s 2s/step - loss: 0.4475 - accuracy: 0.9628 - val_loss: 1.8482 - val_accuracy: 0.8056

In [15]:
# evaluate the network again
preds = model.predict(testx, batch_size=32)
print(classification_report(testy.argmax(axis=1),
                            preds.argmax(axis=1),
                            target_names=classNames))

              precision    recall  f1-score   support

         3T3       1.00      1.00      1.00       156
       MG 63       1.00      1.00      1.00       129
        hASC       1.00      1.00      1.00       147

    accuracy                           1.00       432
   macro avg       1.00      1.00      1.00       432
weighted avg       1.00      1.00      1.00       432



In [26]:
# save serialised model
model.save(args['model'])